In [1]:
from pymol import cmd
import pandas as pd
import torch
import re
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
cmd.load('E:/Coding/jupyter_root/bioinfo_code/cox2.pdb')
cmd.load('E:/Coding/jupyter_root/bioinfo_code/cox1.pdb')

#select cox2_maskseq, cox2 and (resi -104 | resi 106-) and name CA
print(cmd.select('cox2_maskseq', 'cox2 and (resi -104 | resi 106-) and name CA'))
#select cox1_maskseq, (cox1 and resi 34-77 | resi 79-583) and name CA
print(cmd.select('cox1_maskseq', 'cox1 and (resi 34-77 | resi 79-583) and name CA'))

549
549


In [3]:
cmd.align('cox1_maskseq', 'cox2_maskseq')
cox2_coor = cmd.get_coords('cox2_maskseq')
cox1_coor = cmd.get_coords('cox1_maskseq')

print(cox2_coor)

[[ 50.242  30.738  42.592]
 [ 53.561  29.109  43.6  ]
 [ 51.796  26.899  46.169]
 ...
 [ 30.03  -11.195  59.874]
 [ 28.349 -14.242  58.346]
 [ 28.147 -17.361  60.576]]


In [4]:
print(cox1_coor)

[[ 51.500004  29.938644  42.111294]
 [ 54.373127  28.247227  44.00754 ]
 [ 51.952248  26.33754   46.25969 ]
 ...
 [ 29.694786 -12.060076  60.234985]
 [ 27.99388  -14.998503  58.511497]
 [ 27.980455 -18.480463  60.12704 ]]


In [5]:
def get_dist(coor1, coor2, cutoff=15, thresholds=[0.5, 1.0, 2.0, 4.0]):
    
    tensor_coor1 = torch.tensor(coor1, device=device)
    tensor_coor2 = torch.tensor(coor2, device=device)
    dist_obj1 = torch.cdist(tensor_coor1, tensor_coor1)
    dist_obj2 = torch.cdist(tensor_coor2, tensor_coor2)
    mask = (dist_obj1<cutoff)& (dist_obj1 > 0)
    diff = torch.abs(tensor_coor1 - tensor_coor2)

    scores = sum((diff < t) for t in thresholds)/len(thresholds)
    scores = scores.sum(dim = -1) / mask.sum(dim = -1).clamp(min=1)
    return scores.mean()

s = get_dist(cox2_coor, cox1_coor)
s

tensor(0.0526)

In [17]:
def lddt_masked(true_coords, model_coords, cutoff=15.0, thresholds=[0.5, 1.0, 2.0, 4.0]):
    """
    true_coords, model_coords: (N, 3)
    valid_mask: (N,) bool tensor, True 表示该位置有有效坐标
    """
    true_coords = torch.tensor(true_coords, device=device)
    model_coords = torch.tensor(model_coords, device=device)

    d_true = torch.cdist(true_coords, true_coords)
    d_model = torch.cdist(model_coords, model_coords)

    mask = (d_true < cutoff) & (d_true > 0)
    diff = torch.abs(d_true - d_model)

    scores = sum((diff < t) & mask for t in thresholds) / len(thresholds)
    scores = scores.sum(dim=-1) / mask.sum(dim=-1).clamp(min=1)
    return scores.mean()
l = lddt_masked(cox2_coor, cox1_coor)
l

tensor(0.8922)